In [ ]:
#imports
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
protocol = 'BLE'

In [ ]:
#data parsing
#NOTE: would not recommend using original dataset
#since ground truths labelling seems to be off

#scenario 3
# Scenario 1 was a 6.0 x 5.5 m meeting room. 
# This area was a high noise environment as additional transmitting devices were placed 
# around the environment in order to create interference in the signals. 
# There were 16 fingerprints gathered with a larger distance selected between the points. 

fingerprints = {}
for i in range(1, 40+1):
  fingerprints[i] = pd.read_csv( str(i)+ '.txt', names=['Node', 'RSSI'], delimiter=':')



In [ ]:
ground_truth_coords = pd.read_excel('Database_Scenario3.xlsx', sheet_name=protocol)


In [ ]:
def ComputeDistance(rssi):
  return 10**((-RSSI_d0+rssi)/(10*N))

def GetDistance(df):
  arr = {'Node A':[], 'Node B':[], 'Node C':[]}
  for index, datapt in df.iterrows():
    dist = datapt['RSSI']
    dist = ComputeDistance(datapt['RSSI'])
    arr[datapt['Node']].append(dist)
  arr_means = {}#{'Node A':('mean':0, 'std':0), 'Node B':('mean':0, 'std':0), 'Node C':('mean':0, 'std':0)}
  for node in arr:
    lis = np.array(arr[node])
    arr_means[node] = {'mean':np.mean(lis), 'std':np.std(lis)}
    
  return arr_means

def LocateDevice():
  distances = {}
  for dev in fingerprints:
      distances[dev] = GetDistance(fingerprints[dev])
  return distances 

distances_1to5 = LocateDevice()

#trilateration
def trilaterate(location_id, distance_node, Ax1, Ay1, Bx2, By2, Cx3, Cy3):
  # Trilateration parameters calculation
  # Calculate A, B, C, D, E, and F 
  r1 = distance_node['Node A']['mean']
  r2 = distance_node['Node B']['mean']
  r3 = distance_node['Node C']['mean']

  A = Bx2*2 - Ax1*2
  B = 2*By2 - 2*Ay1
  C = r1**2 - r2**2 - Ax1**2 + Bx2**2 - Ay1**2 + By2**2
  D = 2*Cx3 - 2*Bx2
  E = 2*Cy3 - 2*By2
  F = r2**2 - r3**2 - Bx2**2 + Cx3**2 - By2**2 + Cy3**2
  xpt = (C*E - F*B) / (E*A - B*D)
  ypt = (C*D - A*F) / (B*D - A*E)
  
  # print(location_id, ': X=', xpt, ', Y=', ypt )
  return(xpt, ypt)

def trilaterating(Ax1, Ay1, Bx2, By2, Cx3, Cy3):

  RSSI_d0 = -65
  N = (-71-(-65))/(10*np.log10(2))

  x_coords = []
  y_coords = []
  labels = list(np.arange(39)+1)
  for location in distances_1to5:
    x_location, y_location = trilaterate(location, distances_1to5[location],Ax1, Ay1, Bx2, By2, Cx3, Cy3)
    x_coords.append(x_location)
    y_coords.append(y_location)
  # plotting error:
  err = 0.0
  for i in range(0, 40):
    x = ground_truth_coords['x'][i]
    y = ground_truth_coords['y'][i]
    err+= (abs(x_coords[i] -x)**2 + abs(y_coords[i]-y)**2)**(0.5)

  err /= 40.0
  print(err)

# Ax1 =0
# Ay1 =0

# # AP2
# Bx2 = 4.8125 
# By2 = 2.492

# # AP3
# Cx3 =9.625 
# Cy3 = 0


trilaterating(0,0,9.625,0,4.8125,2.493)



4.26959478605338
